# *Dynamic Routing*
<img src="https://www.kvc.org/wp-content/uploads/2015/08/brain-600px-600x372.jpg" width="320" />

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [ ]:
import pathlib

import IPython
import ipywidgets as widgets

import np_logging
import np_session
import np_services
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.dynamic_routing as DR
import npc_shields

from np_services.resources.zro import ZroError 
import contextlib

np_logging.getLogger()

np_workflows.elapsed_time_widget()

***
## Quiet mode
on  
*default*
- exception details are hidden
- regular messages displayed (log level = INFO)

off
- full exception details with traceback
- extra messages displayed (log level = DEBUG)

In [ ]:
np_workflows.quiet_mode_widget()

***
## Launch apps via RSC
[optional]

In [ ]:
import contextlib
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Select workflow
Re-run this cell if mouse ID is changed

In [ ]:
selection = DR.workflow_select_widget(mouse)

***
## Set MTrain stage for mouse
[optional] skip this for ephys

In [ ]:
np_workflows.mtrain_widget(mouse)

***
## Generate new session
Re-run this cell if workflow is changed above.

Check mouse ID and session are correct: this cell will lock them in!

In [ ]:
experiment: np_workflows.DynamicRoutingExperiment = DR.new_experiment(mouse, user, selection.workflow)
session: np_session.Session = experiment.session

***
## Set task
- select a preset task variant
- modify in the text box as needed (right) - hit enter to apply

In [ ]:
np_workflows.task_select_widget(experiment)

Double-check task until we have confidence in the new widget!

In [ ]:
print(f'Task is currently set to: \n\n\t{experiment.task_name}')

## Check/modify github hash

In [ ]:
experiment.commit_hash, experiment.github_url, experiment.use_github

Verify that the required script can be found on github:

In [ ]:
if experiment.use_github:
    assert experiment.camstim_script.exists(), f"{experiment.camstim_script = } could not be accessed - don't continue!"
print('Script found on github - all clear!')

***
## Set up some aliases for the workflow

Manually assign `True` or `False` if you need to alter the workflow

Note: overriding these bools will only affect which lines run in the notebook.
Other experiment code may check `experiment.is_ephys` etc. behind the scenes.

In [ ]:
hab: bool = experiment.is_hab
opto: bool = experiment.is_opto     # 'opto' in task name: opto stim will run during task trials
pretest: bool = experiment.is_pretest
ephys: bool = experiment.is_ephys or pretest  # probes will be inserted: (ephys and opto) is possible
optotagging: bool = experiment.is_optotagging or pretest

***
## Pretest
Checks before running

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
if ephys:
    np_workflows.check_openephys_widget()

---
### *Temporary: skip OpenEphys while NP.3-Acq isn't available*

In [ ]:
experiment.services = experiment.services[:6]
experiment.__class__.recorders = experiment.recorders[:2]

Stop acquiring in OpenEphys and set the recording name to:

In [ ]:
print(str(experiment.session))

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [ ]:
assert experiment.task_name, "task name has not been set - assign manually in cell above"
experiment.initialize_and_test_services()

***
## Run sound test

In [ ]:
experiment.run_sound_test()

---
## MouseDirector: extend lick spout and set position for mouse
- so it doesn't fly out to an unknown position when the mouse is on the stage

***
## Dip probes

In [ ]:
if ephys:
    np_workflows.dye_widget(session.npexp_path)
    DR.photodoc_widget(session, 'pre_experiment_surface_image')

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
experiment.log('Mouse on stage')
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

In [ ]:
DR.photodoc_widget(session, 'brain_surface_image')

***
## Probe insertion
*re-run the cell after modifying `shield_name` or `experiment_day`*

In [ ]:
if ephys and not pretest:
    
    save_paths = (
        session.npexp_path / 'insertions.json',
        pathlib.Path(f'//allen/programs/mindscope/workgroups/dynamicrouting/ben/implants/insertion_records/{session.folder}.json'),
    )
    widget = npc_shields.get_insertion_widget(
        save_paths=save_paths,
        session=session.folder,
        # -----------------------------------------------------------------------
        shield_name='',         # <-- UPDATE THIS: '2002', '2005', '2006', etc.
        experiment_day=0,       # <-- UPDATE THIS: 1, 2, 3, 4, ...
        # -----------------------------------------------------------------------
    )
    
    display(widget)

In [ ]:
if ephys or opto:
    np_workflows.isi_widget(mouse.lims)

---
## Extra advance & retract each probe
- use NewScale GUI to advance an extra 100 $\mu m$ at 200 $\mu m/s$, then reverse 100 $\mu m$ at the same rate

***
## Photodoc before advancing probes

In [ ]:
if ephys or opto:
    DR.photodoc_widget(session, 'pre_insertion_surface_image')

***
## Settle timer with dark screen (screen should be closed)
### *Turn on laser & configure OptoGUI while waiting...*

In [ ]:
experiment.set_dark_desktop_on_stim()
if ephys:
    experiment.log('settle timer started')
    np_workflows.print_countdown_timer(minutes=.1 if experiment.is_pretest else 20)
    experiment.log('settle timer finished')

### Confirm OptoGUI params file can be found
*opto stimuli/tagging won't be able to run without it!*

In [ ]:
if ephys:
    display('optotagging', experiment.optotagging_params)
if opto:
    display('opto', experiment.opto_params)

***
## Photodoc after probes settled, before experiment

In [ ]:
if ephys or opto:
    DR.photodoc_widget(session, 'post_insertion_surface_image')

In [ ]:
np_workflows.pre_stim_check_widget()

***
## Start devices recording

In [ ]:
experiment.start_recording()

***
## Without lick spout
### mapping | optotagging | spontaneous

In [ ]:
np_services.MouseDirector.get_proxy().retract_lick_spout()

if ephys or hab:
    experiment.run_mapping()

if optotagging:
    experiment.run_optotagging()
        
if ephys:
    experiment.run_spontaneous()

*** 
## With lick spout
- set mouse offset
- flush line, check water delivery

In [ ]:
np_services.MouseDirector.get_proxy().extend_lick_spout()

### spontaneous + rewards | task | spontaneous + rewards
Check lickspout is extended before starting!

In [ ]:
if ephys or hab:
    experiment.run_spontaneous_rewards()
    
experiment.run_task()

if ephys:
    experiment.run_spontaneous_rewards()

***
## Without lickspout
### optotagging

In [ ]:
np_services.MouseDirector.get_proxy().retract_lick_spout()

if ephys:
    experiment.run_spontaneous()

if optotagging:
    experiment.run_optotagging()

***
## Stop recording

In [ ]:
experiment.stop_recording_after_stim_finished()

np_services.MouseDirector.get_proxy().retract_lick_spout()
experiment.reset_desktop_on_stim()

---
### *Temporary: manually stop recording in Open Ephys*

*** 
## Surface channel recording (experimental)

Run this to make sure the previous recording gets copied after the surface channel recording:

In [ ]:
np_services.OpenEphys.finalize()

In [ ]:
if ephys and not pretest:
    import requests, json, time

    open_ephys = np_services.OpenEphys

    NEUROPIX_PXI_PROCESSOR = 100 # <- check number in the `Graph` tab in the GUI (right, above `LFP`)

    # set all probes to use middle channel bank, and set tip-reference 
    channels = range(385, 384*2 + 1) # c++ 1-indexed
    for dock in (1,):
        for slot in (2, 3):
            for port in (1, 2, 3,):
                msgs = []
                msgs.append({"text": f"NP SELECT {slot} {port} {dock} {' '.join(map(str, channels))}"})
                msgs.append({"text": f"NP REFERENCE {slot} {port} {dock} {'TIP'}"})
                for msg in msgs:
                    response = requests.put(u := (open_ephys.url(open_ephys.Endpoint.processors) + f"/{NEUROPIX_PXI_PROCESSOR}/config"), json.dumps(msg))
                    time.sleep(0.2)

In [ ]:
if ephys and not pretest:
    open_ephys.set_folder(f"{session}_surface_channels")

In [ ]:
## this shouldn't be necessary but in case start fails:
# open_ephys.set_state(open_ephys.State.acquire) 
if ephys and not pretest:
    open_ephys.start()
    time.sleep(60 * 5)
    open_ephys.stop()

In [ ]:
if ephys and not pretest:
    # reset all probes to use lower channel bank, and set tip-reference 
    channels = range(1, 384 + 1) # c++ 1-indexed
    for dock in (1,):
        for slot in (2, 3):
            for port in (1, 2, 3,):
                msgs = []
                msgs.append({"text": f"NP SELECT {slot} {port} {dock} {' '.join(map(str, channels))}"})
                msgs.append({"text": f"NP REFERENCE {slot} {port} {dock} {'TIP'}"})
                for msg in msgs:
                    response = requests.put(u := (open_ephys.url(open_ephys.Endpoint.processors) + f"/{NEUROPIX_PXI_PROCESSOR}/config"), json.dumps(msg))
                    time.sleep(0.2)

***
## Before removing probes

In [ ]:
if ephys or opto:
    DR.photodoc_widget(session, 'post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
if ephys or opto:
    DR.photodoc_widget(session, 'post_experiment_surface_image')

### Go back and check insertion records are correct in implant widget

***
## After raising cartridge

In [ ]:
np_workflows.finishing_checks_widget()

## Record mouse off stage time

In [ ]:
experiment.log('Mouse off stage')

***

## Finalize

In [ ]:
experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

### copy all raw dataexcept ephys to /allen 

In [ ]:
experiment.copy_data_files()

### run and save QC notebook in current working directory 

In [ ]:
with contextlib.suppress(Exception):
    import npc_sessions
    npc_sessions.write_qc_notebook(
        session_path_or_id=experiment.session.npexp_path.as_posix(),
        save_path=pathlib.Path.cwd(),
        ephys_record_node_dirs=tuple(
            next(path.glob('Record Node*')).as_posix()
            for path in np_services.OpenEphys.data_files
            if experiment.session.folder in path.name
        ),
    )

In [ ]:
import np_jobs
experiment.copy_workflow_files()
# np_jobs.PipelineQCQueue().add_or_update(session, priority=99)
experiment.copy_ephys()